In [ ]:
pip install -q yahoo_fin

In [2]:
mkdir -p data results log

In [ ]:
pip install  git+https://github.com/ahsank/runml@v0.0.5#egg=runml

In [4]:
import runml.pipeline
from runml import pipeline,findata
from google.colab import drive
#from importlib import reload
# reload(runml.pipeline)

Run following if like to save results in drive

In [5]:
drive.mount('/content/drive')
!cp  /content/drive/MyDrive/colab/results/* ./results/

Mounted at /content/drive


In [6]:
import numpy as np
# profit factor 2 = 50% of predicted gain due to early profit taking
def addAlloc(df, stop_loss, profit_factor=2):
  df['Alloc'] = df['Accuracy']/stop_loss - (1-df['Accuracy'])*profit_factor/abs(df['Gain'])
  df['Alloc'] = np.where(df['Alloc'] < 0, 0, df['Alloc'])
  return df

In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
pending = ['PAR']
tickers = ['ABNB', 'ACLS' ,'AI', 'AMBA', 'APP', 'AYX',
           'BILL', 'BMBL', 'CELH', 'CFLT', 'CHGG', 'CRWD', 'CVNA',
           'DASH', 'DBX', 'DDOG', 'DOCN', 'DOCS', 'DOCU', 'DT', 'DXCM',
           'ENPH', 'ESTC', 'ETSY', 'EXPE', 'FIVE', 'FOUR',
           'GFS', 'GTLB', 'GLBE', 'GOGO',
           'INDI', 'INMD', 'INTA', 'IOT', 'IRDM', 'JKS', 'LYFT',
           'MBLY', 'MDB',  'MNDY', 'MNST', 'MPWR',  'MXL',
           'MTCH',  'NVCR', 'NTNX', 'OKTA', 'OLED',
           'PERI', 'PANW',  'PAYX', 'PD', 'PLUG', 'PI', 'PINS', 'PUBM',
           'RBLX', 'SMCI', 'SNAP', 'SNOW', 'SPLK', 'SQ', 'SPOT',
           'TEAM', 'TDOC', 'TOST', 'TRIP', 'TTD', 'TWLO',
           'U','UI', 'UBER', 'UPWK', 'WOLF', 'VEEV', 'Z', 'ZM', 'ZS']
lossfn = "huber_loss"
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df  = pipeline.runModelCombined(tickers, 'ipos-12a', mod, False, loss=lossfn)[0]

In [8]:
addAlloc(df, 0.10, 1)
df.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
12,CVNA,RROnly,1514.43,1806.66,3321.09,7.565728,0.962766,42.78,85.040001,0.99,9.590049
33,INTA,RROnly,164.94,184.77,349.71,14.311456,0.973913,41.36,36.529999,-0.12,9.521739
0,ABNB,RROnly,768.33,809.30,1577.63,84.661933,0.965035,137.14,157.229996,0.15,9.417249
70,U,RROnly,548.48,876.08,1424.56,24.002717,0.948387,34.59,52.000000,0.50,9.380645
9,CFLT,RROnly,206.40,424.89,631.29,17.337906,0.948276,22.31,28.650000,0.28,9.298030
22,ESTC,RROnly,1226.06,955.98,2182.04,46.172062,0.957447,111.94,123.820000,0.11,9.187621
28,GTLB,RROnly,379.13,420.00,799.13,28.107307,0.940594,61.56,78.470001,0.27,9.185919
74,WOLF,RROnly,823.83,856.88,1680.71,30.038895,0.936170,33.86,45.939999,0.36,9.184397
76,Z,RROnly,574.42,691.09,1265.51,26.762205,0.936170,53.50,69.459999,0.30,9.148936
39,MDB,RROnly,3924.14,2837.50,6761.63,107.565913,0.941489,391.59,479.309998,0.22,9.148936


In [ ]:
findata.EPOCHS=200
later = [ ]
# https://seekingalpha.com/etfs-and-funds/etf-tables/sectors
tickers1 = ['ARKF', 'ARKK', 'ARKW', 'DAPP', 'DIA', 'DTEC', 'EEM', 'FPX',
            'ICLN', 'IJR', 'IPO', 'IXC', 'IXN', 'IXP', 'IWM', 'IWO', 'IYZ',
            'JETS', 'MGK', 'MGV', 'MTUM',
            'ONLN', 'QQQ', 'SMH', 'SMOG', 'SPY', 'TDIV',
            'VNQ', 'VT', 'VTI', 'VUG', 'WDIV', 'XITK',
            'XLB', 'XLC', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP',
            'XLRE', 'XLU', 'XLV', 'XLY', 'XME', 'XNTK', 'XSW' ]
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df1, res = pipeline.runModelCombined(tickers1, 'etf-11a', mod, True)


In [ ]:
detail = res['ARKK'].final_df

mindate = detail['date'].min()
totalday = (detail['date'].max()-mindate).days
detail['diff'] = ((detail['date']-mindate).dt.days+1)/totalday
detail['sum_profit'] = (detail['buy_profit'] + detail['sell_profit'])/detail['adjclose']
detail['predicted_profit'] = ((detail['adjclose_15']-detail['adjclose'])/detail['adjclose']).abs()
profit_df = detail[detail['sum_profit'] > 0][['sum_profit', 'predicted_profit']]
average_profit = (profit_df['sum_profit']).mean()
excess_profit = (profit_df['sum_profit'] - profit_df['predicted_profit']).mean()
average_loss = -detail[detail['sum_profit'] < 0]['sum_profit'].mean()
detail['has_profit'] = np.where(detail['sum_profit'] > 0, 1 , 0)

accuracy = detail['has_profit'].mean()
weight_accuracy = (detail['has_profit']*detail['diff']).sum()/detail['diff'].sum()
(accuracy, weight_accuracy, average_profit, excess_profit, average_loss)

(0.9202127659574468,
 0.9229203567322334,
 0.09075827015321843,
 0.004420030125699004,
 0.01842253193096099)

In [10]:
addAlloc(df1, 0.10, 1)
df1.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
18,MGK,RROnly,1207.64,566.69,1774.32,110.376022,0.984043,261.46,250.750000,-0.04,9.441489
12,IXN,RROnly,269.99,176.51,446.50,26.466679,0.978723,67.82,63.889999,-0.06,9.432624
14,IWM,RROnly,922.14,576.60,1498.74,96.557606,0.957447,193.23,210.830002,0.09,9.101655
1,ARKK,RROnly,614.99,604.26,1219.26,30.780882,0.930851,47.38,59.180000,0.25,9.031915
5,DTEC,RROnly,232.41,129.26,361.66,22.949856,0.962766,40.68,43.200001,0.06,9.007092
21,ONLN,RROnly,346.33,299.02,645.35,26.974758,0.936170,34.13,39.220001,0.15,8.936170
15,IWO,RROnly,1241.10,838.72,2079.82,134.804310,0.946809,243.19,262.950012,0.08,8.803191
38,XLK,RROnly,773.10,446.21,1219.31,69.450104,0.957447,192.24,181.970001,-0.05,8.723404
24,SMOG,RROnly,718.84,746.19,1465.02,50.915070,0.941489,102.77,109.750000,0.07,8.579027
19,MGV,RROnly,328.86,178.34,507.20,51.985502,0.952128,109.59,104.400002,-0.05,8.563830


In [ ]:
findata.EPOCHS=200
pipeline.IS_VERBOSE = False
# exclude = ['GTSWF', 'CBNT', 'CCL', 'CREX'. 'DTEA'. 'ILAL',
# 'TNXP', 'MRIN', 'MULN', 'AHT',
# 'LILM', 'JMIA', 'RKLB', 'VRAR', 'WISH',]
tickers3 = [ 'AIRI', 'AMSC', 'APRN', 'ACHR', 'AUGX', 'AUR',   'ATOM',
            'BKKT', 'BGSF', 'BLNK', 'BSGM',
            'CARS', 'CLOV', 'CRCT', 'CXM', 'CZOO',
            'DLO',  'DM', 'DMTK', 'DPSI', 'EVGO', 'EXPR', 'FSLY', 'FSR', 'FTCH',
            'GOVX', 'GRAB', 'GTE', 'HIMS', 'HIVE',
            'INLX',  'INVZ', 'JOBY',  'KIND', 'KULR',
            'LAZR', 'LUMN', 'NNOX', 'MAX', 'MTTR',
            'NEPH', 'NSTG', 'OMQS', 'ONDS', 'OPEN', 'OTLY', 'OUST',
            'PAYO', 'PTON', 'REAL', 'REI', 'RIG', 'RKLB', 'RVYL',
            'SFIX', 'SOUN', 'STEM', 'SFT',  'TELL', 'VTEX', 'YEXT', 'WKME']
mod = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
# mod = pipeline.RateReturnOnly(pipeline.AddDayMonth())
df3 = pipeline.runModelCombined(tickers3, 'vols-12b', mod, True)[0]

In [ ]:
# df3.sort_values('Gain', ascending=False)
addAlloc(df3, 0.17, 1)
df3.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
52,RKLB,RROnly,58.91,73.83,132.74,3.865492,0.950704,4.61,5.21000,0.13,5.213180
43,ONDS,RROnly,31.15,58.88,90.04,0.559116,0.892857,1.37,2.60000,0.90,5.133053
24,FTCH,RROnly,266.15,300.13,566.28,1.684773,0.882979,0.64,1.78000,1.78,5.128250
58,TELL,RROnly,43.45,38.42,81.87,0.633339,0.909574,0.68,0.84000,0.24,4.973665
57,SFT,RROnly,25.37,61.34,86.71,0.198132,0.843537,0.03,0.17000,5.22,4.932011
47,PAYO,RROnly,39.08,47.79,86.87,3.634892,0.897959,4.80,6.04000,0.26,4.889648
41,NSTG,RROnly,298.96,482.07,781.03,1.656227,0.840426,0.54,2.01000,2.71,4.884796
20,EVGO,RROnly,45.92,133.31,179.23,2.377537,0.894366,3.49,4.43000,0.27,4.869741
14,CXM,RROnly,74.79,67.89,142.68,7.964617,0.920354,11.22,12.79000,0.14,4.844947
34,KULR,RROnly,28.67,24.15,52.82,0.347572,0.904255,0.28,0.33000,0.19,4.815230


In [ ]:
findata.EPOCHS=200
tickers4 = ['AAPL', 'ACN', 'ADBE', 'AMD', 'AMZN', 'AVGO', 'BA', 'BKNG', 'BRK-B',
            'CAT', 'CDNS', 'CRM', 'COST', 'CSCO', 'DE', 'DHR', 'DIS',
            'GOOGL', 'FDX', 'HD', 'INTC', 'IBM', 'ISRG',
            'META', 'MMC', 'MSFT', 'NKE', 'NVDA', 'NOW',
            'ORCL', 'PYPL', 'QCOM', 'SBUX', 'TMO', 'TSLA', 'TXN', 'UNH', 'UPS']
mod4 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df4 = pipeline.runModelCombined(tickers4, 'mcap-11b', mod4, False)[0]

In [ ]:
# df4.sort_values('Gain', ascending=False)
addAlloc(df4, 0.14, 1)
df4.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
26,NKE,RROnly,727.72,626.20,1353.92,62.190753,0.936170,108.04,128.339996,0.19,6.350984
23,META,RROnly,1816.60,1503.39,3319.99,92.930568,0.957447,353.39,333.480011,-0.06,6.129686
34,TSLA,RROnly,2692.96,2170.05,4863.01,31.991598,0.925532,252.54,283.579987,0.12,5.990375
10,CDNS,RROnly,1160.29,570.43,1730.72,57.544700,0.909574,275.82,249.490005,-0.10,5.592705
35,TXN,RROnly,841.77,581.22,1422.99,86.052421,0.941489,168.24,160.309998,-0.05,5.554711
0,AAPL,RROnly,865.53,501.73,1367.26,58.339881,0.877660,193.60,161.029999,-0.17,5.549347
16,DIS,RROnly,651.03,676.74,1327.77,80.865222,0.898936,91.02,99.879997,0.10,5.410334
27,NVDA,RROnly,2367.26,1606.71,3973.97,55.140231,0.898936,488.30,437.679993,-0.10,5.410334
9,CAT,RROnly,1130.48,766.99,1897.47,87.766133,0.904255,290.36,267.130005,-0.08,5.262158
6,BA,RROnly,1359.22,1260.35,2619.57,100.762304,0.909574,260.44,277.029999,0.06,4.989868


In [ ]:
findata.EPOCHS=200
tickers5 = [ 'AEXAF', 'ALSMY', 'AMSSY', 'ASML', 'ADDDF', 'ADYEY',
            'BAK', 'BASFY', 'BESIY',
            'CAAP', 'CDUAF', 'COTY', 'COVTY', 'CDMGF', 'CPG', 'CX',
            'DBOEY',
            'EADSF', 'EIFZF', 'EDNMF', 'ENGGY', 'EQNR', 'ERIC', 'ERJ','ESEA',
            'FANUY', 'FQVLF', 'GLCNF',  'GLOB', 'HCMLY', 'HDELY',
            'HENKY', 'HYMTF', 'HMC', 'KNNGF', 'MELI', 'MGA',
            'RACE', 'LYB', 'LVMUY',
            'MYTE', 'NHYDY', 'NOK', 'NU', 'NVO', 'NXPI',
            'ONON', 'OTGLY', 'PAX', 'POAHF', 'PROSF',
            'RNMBF', 'RYCEF', 'SAFRF', 'SFTBY',
            'ORAN', 'SAP', 'SE', 'SIEGY', 'STLA',
            'TEF', 'TELNF', 'TGLS','THQQF', 'TKAMY', 'TRYIY', 'TSM',
            'VALE', 'VNT', 'YARIY', 'ZURVY']
mod5 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df5 = pipeline.runModelCombined(tickers5, 'gstock-12b', mod5, True)[0]


In [ ]:
addAlloc(df5, 0.14)
df5.sort_values('Alloc', ascending=False).head(30)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
26,FQVLF,RROnly,230.24,226.96,457.20,4.059197,0.936170,8.32,11.950000,0.44,6.396795
50,PROSF,RROnly,481.22,738.88,1220.10,28.393939,0.925532,27.88,34.720001,0.25,6.015198
36,MGA,RROnly,388.86,399.13,787.99,23.045348,0.946809,59.03,66.529999,0.13,5.944587
54,SFTBY,RROnly,217.06,162.03,379.09,13.444299,0.930851,21.22,24.809999,0.17,5.835419
40,MYTE,RROnly,53.41,111.40,164.81,2.461305,0.888889,3.28,4.680000,0.43,5.832410
1,ALSMY,RROnly,22.76,26.40,49.17,1.211703,0.893617,1.29,1.670000,0.30,5.673759
46,ONON,RROnly,146.39,149.12,295.51,16.760453,0.901961,28.03,34.240002,0.22,5.551311
23,ERJ,RROnly,135.77,103.63,239.40,4.471557,0.946809,18.63,17.090000,-0.08,5.433131
2,AMSSY,RROnly,49.85,68.45,118.31,1.775798,0.824468,1.60,2.730000,0.70,5.387538
57,SE,RROnly,1456.04,1334.69,2790.73,41.502114,0.882979,38.58,46.980000,0.22,5.243161


In [ ]:
findata.EPOCHS=200
tickers6 = ['ARGT', 'EWD', 'FLN', 'FM',
            'ECH', 'EWA', 'EWI', 'EWC', 'EWM', 'EWP', 'EWW', 'EWY', 'EWZ',
            'IDX', 'ILF', 'KEMQ', 'THD', 'TUR', 'XCEM']
mod6 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df6 = pipeline.runModelCombined(tickers6, 'cetf-11b', mod6, False)[0]


In [ ]:
addAlloc(df6, 0.10, 1)
df6.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
5,EWA,RROnly,78.78,50.45,129.24,11.232726,0.909574,24.34,22.420000,-0.08,7.965426
16,THD,RROnly,214.04,203.48,417.52,45.113771,0.898936,63.86,67.489998,0.06,7.304965
13,IDX,RROnly,57.25,52.66,109.92,9.776208,0.840426,17.07,18.700001,0.10,6.808511
2,FLN,RROnly,72.47,51.70,124.17,8.794977,0.845745,20.79,18.900000,-0.09,6.743499
12,EWZ,RROnly,151.34,118.96,270.30,16.305043,0.888298,34.96,33.180000,-0.05,6.648936
1,EWD,RROnly,139.16,131.62,270.78,19.663916,0.882979,39.47,41.520000,0.05,6.489362
7,EWC,RROnly,115.29,98.79,214.09,16.683844,0.909574,36.68,35.490002,-0.03,6.081560
15,KEMQ,RROnly,75.87,134.72,210.59,11.916286,0.856383,14.70,15.410000,0.05,5.691489
3,FM,RROnly,68.79,61.51,130.30,17.851691,0.851064,26.30,27.690001,0.05,5.531915
6,EWI,RROnly,95.82,76.64,172.46,16.479318,0.824468,33.93,31.969999,-0.06,5.319149


In [ ]:
findata.EPOCHS=200
review = ['AMR', 'IFF', 'BLK']
tickers7 = ['AAL', 'ADI', 'ALB', 'ANF', 'APO', 'AQN', 'ARCH', 'ARE',
            'BAH', 'BAX', 'BKR', 'BGFV', 'BX', 'BXC',
            'CBT', 'CC', 'CCI', 'CCJ', 'CF', 'CHK', 'CI', 'CIVI', 'CTAS',
            'CG',  'CLS', 'CMI', 'COKE', 'COP', 'CSL',
            'DAL', 'DD', 'DINO', 'DVN', 'DXC', 'EMN',
            'F', 'FCX', 'FICO', 'FIS', 'FLO', 'FMS', 'FSLR', 'GLW', 'GWW',
            'HIW', 'HSII', 'HPE', 'IP', 'IPI', 'IVZ', 'JWN',
            'KD', 'KHC', 'KMB', 'KVUE',
             'M', 'MMM', 'MOD', 'MPC', 'NFG', 'NTR', 'OMC', 'OLN', 'OSK',
            'PARA', 'PCAR', 'PEP', 'PETS', 'PNR', 'POR', 'POWL', 'PSTL', 'PXD',
            'RCL', 'RH', 'ROP',
            'SAFE', 'SCCO', 'SPTN', 'SPG', 'SRE', 'STLD', 'STRL', 'SWK',
            'T', 'TCN', 'TEX', 'TGT', 'TDG', 'TMUS','TWI', 'UAL', 'URI',
            'VRTX', 'VZ', 'WDC', 'WHR', 'WSM', 'XOM', 'YORW']

mod7 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df7 = pipeline.runModelCombined(tickers7, 'val-12b', mod7, True)[0]

In [ ]:
addAlloc(df7, 0.10, 1)
df7.sort_values('Alloc', ascending=False).head(50)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
91,UAL,RROnly,394.60,366.50,761.10,21.001723,0.962766,41.26,48.220001,0.17,9.408636
13,BXC,RROnly,805.55,528.34,1333.89,5.395873,0.957447,113.31,99.849998,-0.12,9.219858
89,TMUS,RROnly,640.01,377.03,1017.05,75.458418,0.962766,160.33,150.000000,-0.06,9.007092
76,SAFE,RROnly,581.17,548.00,1129.17,16.470097,0.914894,23.40,33.560001,0.43,8.951014
2,ALB,RROnly,2017.45,1512.58,3530.04,54.163776,0.946809,144.48,157.559998,0.09,8.877069
88,TDG,RROnly,4069.39,2610.40,6679.79,240.086988,0.925532,1011.60,879.119995,-0.13,8.682488
67,PETS,RROnly,84.98,153.78,238.76,7.411406,0.893617,7.56,9.540000,0.26,8.527005
64,PARA,RROnly,304.13,242.19,546.32,11.070234,0.877660,14.79,20.459999,0.38,8.454647
12,BX,RROnly,695.75,539.54,1235.29,33.067304,0.909574,130.92,113.040001,-0.14,8.449848
90,TWI,RROnly,104.58,77.27,181.85,1.410000,0.914894,14.88,16.639999,0.12,8.439716


In [ ]:
findata.EPOCHS=200
tickers8 = [ 'GLD', 'ISHG', 'PDBC', 'SLV', 'SOYB',
            'TIP', 'TLT', 'UUP', 'VTIP', 'WEAT']
mod8 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df8 = pipeline.runModelCombined(tickers8, 'com-12a', mod8, False)[0]

In [ ]:
addAlloc(df8, 0.10, 1)
df8.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
9,WEAT,RROnly,26.27,35.65,61.92,5.009887,0.829787,5.88,6.440000,0.10,6.595745
3,SLV,RROnly,93.10,95.85,188.95,11.602837,0.898936,21.21,21.980000,0.04,6.462766
2,PDBC,RROnly,41.27,32.98,74.26,6.411562,0.851064,13.38,14.030000,0.05,5.531915
6,TLT,RROnly,218.09,395.29,613.38,83.496706,0.851064,96.29,101.290001,0.05,5.531915
5,TIP,RROnly,103.74,118.33,222.07,94.620934,0.861702,106.77,110.589996,0.04,5.159574
1,ISHG,RROnly,89.06,96.73,185.79,63.533294,0.909574,70.99,72.370003,0.02,4.574468
4,SOYB,RROnly,62.89,50.77,113.66,13.666469,0.813830,26.18,27.559999,0.05,4.414894
0,GLD,RROnly,466.75,345.81,812.56,140.515087,0.845745,189.35,187.589996,-0.01,0.000000
7,UUP,RROnly,27.30,28.85,56.15,22.577450,0.861702,27.41,27.230000,-0.01,0.000000
8,VTIP,RROnly,27.46,14.34,41.80,40.277506,0.803191,47.38,46.849998,-0.01,0.000000


In [ ]:
findata.EPOCHS=200
tickers9 = [ 'ADA-USD', 'AVAX-USD', 'BTC-USD', 'DOT-USD', 'DOGE-USD',
            'ETH-USD', 'LTC-USD', 'MATIC-USD', 'SOL-USD', 'XRP-USD']
mod9 = pipeline.RateReturnOnly(
    pipeline.FeatureSeq([pipeline.AddDayMonth(), pipeline.AddVWap(), pipeline.AddMA(200)]))
df9 = pipeline.runModelCombined(tickers9, 'cry-1a', mod9, True)[0]

In [ ]:
addAlloc(df9, 0.14, 1)
df9.sort_values('Alloc', ascending=False)

,Ticker,Name,Buy,Sell,Total,Error,Accuracy,Last,Predicted,Gain,Alloc
3,DOT-USD,RROnly,115.68,253.53,369.21,4.519168,0.808511,7.38,13.470000,0.82,5.541552
7,MATIC-USD,RROnly,15.54,12.11,27.64,0.393315,0.840426,0.84,1.050000,0.24,5.338146
1,AVAX-USD,RROnly,595.89,677.40,1273.29,11.362818,0.776596,36.31,66.290001,0.83,5.277951
6,LTC-USD,RROnly,952.51,1499.68,2452.19,49.814025,0.824468,65.63,83.440002,0.27,5.238940
2,BTC-USD,RROnly,203385.20,258900.06,462285.26,16771.558718,0.856383,43957.17,38509.941406,-0.12,4.920213
5,ETH-USD,RROnly,18804.04,21431.09,40235.13,1074.806445,0.882979,2244.79,2416.909912,0.08,4.844225
4,DOGE-USD,RROnly,1.10,1.82,2.92,0.062401,0.728723,0.08,0.130000,0.63,4.774569
0,ADA-USD,RROnly,4.49,10.85,15.35,0.284068,0.728723,0.53,0.820000,0.53,4.693325
8,SOL-USD,RROnly,1334.29,684.71,2019.00,13.895759,0.760638,99.20,120.570000,0.22,4.345123
9,XRP-USD,RROnly,5.49,6.90,12.39,0.341776,0.755319,0.57,0.680000,0.19,4.107343


In [11]:
 # !cp -ru results /content/drive/MyDrive/colab
 !rsync -av --ignore-existing results/* /content/drive/MyDrive/colab/results/
 ! find /content/drive/MyDrive/colab/ -type f -mtime +90 -delete -print

sending incremental file list
etf-1a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5

sent 3,221,641 bytes  received 35 bytes  6,443,352.00 bytes/sec
total size is 192,745,920  speedup is 59.83
/content/drive/MyDrive/colab/results/ipos-10a-test-adjclose-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/ipos-10a-test-high-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/ipos-10a-test-low-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-wadj-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/ipos-10a-sh-1-sc-1-sbd-0-seq-50-step-15-wRROnly-wdm-wvwap-wma-adjclose-200-model-huber_loss-adam-LSTM-layers-2-units-256.h5
/content/drive/MyDrive/colab/results/vols-10a-t